In [35]:
import pandas as pd
data = pd.read_excel(r'C:\Users\cxy\OneDrive\桌面\回国\实习\7-30\东北战区医院基础信息核对&科室信息收集表-20240730.xlsx',sheet_name='辽宁')
df = data.reset_index().drop(columns='index')
df['医院名称'] = df['医院名称'].astype('string')
df

,医院名称
0,沈阳林济中医院
1,海城正骨医院
2,海城普康医院
3,海城骨伤病医院
4,海城仁济医院
...,...
1410,营口站前东桥血栓病医院
1411,营口站前建丰医院
1412,营口市妇幼保健计划生育服务中心
1413,鲅鱼圈市中心医院


In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from fuzzywuzzy import fuzz
import pandas as pd
import time

def setup_driver():
    path_to_chromedriver = r"D:\chromedriver-win64\chromedriver-win64\chromedriver.exe"
    service = Service(executable_path=path_to_chromedriver)
    options = webdriver.ChromeOptions()
    options.headless = True
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def fetch_hospital_info(query_name, driver):
    url = f'https://www.dayi.org.cn/search?keyword={query_name}'
    driver.get(url)
    time.sleep(5)

    results = driver.find_elements(By.CSS_SELECTOR, '.title-left a')
    top_results = results[:3]  # 取前五个结果
    #print(top_results)
    best_match = None
    highest_score = 0

    for result in top_results:
        hospital_name = result.text
        #print(hospital_name)
        score = fuzz.ratio(query_name, hospital_name)
        if score > highest_score:
            highest_score = score
            best_match = result
            #print(highest_score)

    if best_match and highest_score > 60:  # 可以根据需要调整阈值
        hospital_name = best_match.text
        href = best_match.get_attribute('href')
        e_id = href.split('/')[-1].split('.')[0]
        detail_url = f'https://www.dayi.org.cn/hospital/{e_id}.html'
        return query_name, hospital_name, detail_url
    else:
        return query_name, '未找到匹配的医院信息', '未找到链接'

driver = setup_driver()
#results=fetch_hospital_info("大庆油田总医院集团南区医院",driver)
results = [fetch_hospital_info(name, driver) for name in df['医院名称']]
driver.quit()  # 确保浏览器关闭
#results
# 创建结果 DataFrame 并保存
results_df = pd.DataFrame(results, columns=['查询名称', '搜索到的医院名称', 'URL'])
results_df

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.73)
Stacktrace:
	GetHandleVerifier [0x00007FF7C1A7EEA2+31554]
	(No symbol) [0x00007FF7C19F7ED9]
	(No symbol) [0x00007FF7C18B872A]
	(No symbol) [0x00007FF7C188D995]
	(No symbol) [0x00007FF7C19344D7]
	(No symbol) [0x00007FF7C194C051]
	(No symbol) [0x00007FF7C192CDD3]
	(No symbol) [0x00007FF7C18FA33B]
	(No symbol) [0x00007FF7C18FAED1]
	GetHandleVerifier [0x00007FF7C1D88B1D+3217341]
	GetHandleVerifier [0x00007FF7C1DD5AE3+3532675]
	GetHandleVerifier [0x00007FF7C1DCB0E0+3489152]
	GetHandleVerifier [0x00007FF7C1B2E776+750614]
	(No symbol) [0x00007FF7C1A0375F]
	(No symbol) [0x00007FF7C19FEB14]
	(No symbol) [0x00007FF7C19FECA2]
	(No symbol) [0x00007FF7C19EE16F]
	BaseThreadInitThunk [0x00007FFF503D257D+29]
	RtlUserThreadStart [0x00007FFF51A0AF28+40]


In [37]:
results_df = pd.DataFrame(results, columns=['查询名称', '搜索到的医院名称', 'URL'])
results_df

,查询名称,搜索到的医院名称,URL
0,大安市中医院,未找到匹配的医院信息,未找到链接
1,大安市人民医院,会理市人民医院,https://www.dayi.org.cn/hospital/1155606.html
2,大安爱尔眼科医院,未找到匹配的医院信息,未找到链接
3,大安市第四人民医院,未找到匹配的医院信息,未找到链接
4,大安市第一人民医院,未找到匹配的医院信息,未找到链接
...,...,...,...
819,长春爱尔眼科医院,未找到匹配的医院信息,未找到链接
820,长春凯旋康复医院,未找到匹配的医院信息,未找到链接
821,长春长庚耳鼻喉医院,未找到匹配的医院信息,未找到链接
822,长春白求恩口腔医院,未找到匹配的医院信息,未找到链接


In [38]:
results_df1=results_df[results_df['搜索到的医院名称']!='未找到匹配的医院信息']
results_df1


,查询名称,搜索到的医院名称,URL
1,大安市人民医院,会理市人民医院,https://www.dayi.org.cn/hospital/1155606.html
6,白城市中心医院,白城中心医院,https://www.dayi.org.cn/hospital/2894.html
27,白城中心医院,白城中心医院,https://www.dayi.org.cn/hospital/2894.html
28,白城中医院,白城市中医院,https://www.dayi.org.cn/hospital/2952.html
29,白城市第三人民医院,南充市高坪区第三人民医院,https://www.dayi.org.cn/hospital/1157145.html
...,...,...,...
810,长春市中心医院,长春市中心医院,https://www.dayi.org.cn/hospital/1156164.html
811,吉林省肿瘤医院,吉林省肿瘤医院,https://www.dayi.org.cn/hospital/1030739.html
813,农安县中医院,平原县中医院,https://www.dayi.org.cn/hospital/1157183.html
815,吉林省一汽总医院,一汽总医院,https://www.dayi.org.cn/hospital/1558.html


In [39]:
results_df1.to_excel(r'C:\Users\cxy\OneDrive\桌面\回国\实习\7-30\辽宁适配医院url.xlsx')

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def setup_driver():
    path_to_chromedriver = r"D:\chromedriver-win64\chromedriver-win64\chromedriver.exe"
    service = Service(executable_path=path_to_chromedriver)
    options = webdriver.ChromeOptions()
    # 如果你想运行无头模式（不显示浏览器窗口），可以取消下一行的注释
    options.headless = True
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def fetch_dean_name(url):
    #driver = setup_driver()
    driver.get(url)
    #time.sleep(5)
    dean_name = "院长信息未找到"  # 默认值，以防无法获取院长名称
    try:
        # 使用WebDriverWait来等待页面元素加载完成
        wait = WebDriverWait(driver, 5)  # 最多等待10秒
        # XPath to locate the dean's name following the "医院院长" label
        dean_xpath = "//p[contains(text(), '医院院长')]/following-sibling::span[@class='short-field-content']"
        dean_info = wait.until(EC.visibility_of_element_located((By.XPATH, dean_xpath)))
        dean_name = dean_info.text.strip()  # 获取院长的名字
    except Exception as e:
        print(f"Error: {e}")

    return dean_name


import pandas as pd

# 假设 `results_df` 已经存在并包含了 '查询名称', '搜索到的医院名称', 'URL' 列
# 假设 hospital_urls 是包含所有要查询的医院 URL 的列表
#hospital_urls = results_df['URL'].tolist()  # 从 results_df 获取所有的 URL
driver = setup_driver()
# 获取所有院长的名字，这一步可能需要一些时间因为要加载每一个页面
dean_names = [fetch_dean_name(url) for url in results_df1['URL']]
driver.quit()
# 将院长名字添加到现有的 DataFrame 中
results_df1['院长名称'] = dean_names  # 添加院长名称作为新的列

# 显示结果
results_df1



In [ ]:
results_df1.to_excel(r'C:\Users\cxy\OneDrive\桌面\回国\实习\7-30\吉林适配医院院长清单.xlsx')